# FuzConvTrue

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libraries

In [2]:

import numpy as np
import random
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from keras.layers import Activation
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Lambda, Layer, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, SpatialDropout2D, Concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D, BatchNormalization
from keras.layers.core import Reshape
from keras import optimizers
from tensorflow.keras import regularizers
from keras import Input, Model
from time import time
import time as tm
from keras.initializers import Constant, RandomNormal, glorot_normal
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.utils import plot_model

## 30 SRM filters for preprocessing and the activation function

In [3]:
################################################## 30 SRM FILTERS
srm_weights = np.load('drive/MyDrive/Numfiles/SRM.npy') 
biasSRM = np.ones(30)
#print (srm_weights.shape)
################################################## TANH ACTIVATION FUNCTION
T3 = 3;
def Tanh3(x):
    tanh3 = K.tanh(x)*T3
    return tanh3
##################################################

## Architecture

In [5]:
def FuzConvTrue(img_size=256):
    inputs = Input(shape=(img_size, img_size))

    Layers = tf.expand_dims(inputs, axis=-1)  #  channel dimension

    Layers  = Conv2D(96, kernel_size=(11, 11), strides=(4, 4), padding='valid', activation='relu')(Layers )
    Layers  = AveragePooling2D(pool_size=(3, 3), strides=(2, 2))(Layers )

    Layers  = Conv2D(256, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='relu')(Layers )
    Layers  = AveragePooling2D(pool_size=(3, 3), strides=(2, 2))(Layers )

    Layers  = Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(Layers )

    Layers  = Conv2D(384, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(Layers )

    Layers  = Conv2D(256, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(Layers )
    Layers  = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(Layers )

    Layers  = Flatten()(Layers )
    Layers  = Dense(4096, activation='relu')(Layers )
    Layers  = Dense(2, activation='softmax')(Layers )

    model = Model(inputs=inputs, outputs=Layers )
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

model = FuzConvTrue()
model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256)]        0         
                                                                 
 tf.expand_dims_1 (TFOpLambd  (None, 256, 256, 1)      0         
 a)                                                              
                                                                 
 conv2d_5 (Conv2D)           (None, 62, 62, 96)        11712     
                                                                 
 average_pooling2d_2 (Averag  (None, 30, 30, 96)       0         
 ePooling2D)                                                     
                                                                 
 conv2d_6 (Conv2D)           (None, 30, 30, 256)       614656    
                                                                 
 average_pooling2d_3 (Averag  (None, 14, 14, 256)      0   

## Defining different functions to work with the architecture

In [6]:

def train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size, epochs, initial_epoch=0, model_name=""):
    start_time = time()
    log_dir = "/content/drive/My Drive/FuzConvSteganalysis/trained_models/" + model_name + "_" + "{}".format(time())
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir)
    filepath = log_dir + "/saved-model-{epoch:02d}-{val_accuracy:.2f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=False, mode='max')
    model.reset_states()
    history = model.fit(X_train, y_train, epochs=epochs, 
                        callbacks=[tensorboard,checkpoint], 
                        batch_size=batch_size, validation_data=(X_valid, y_valid), initial_epoch=initial_epoch)

    metrics = model.evaluate(X_test, y_test, verbose=0)
    results_dir = "/content/drive/My Drive/FuzConvSteganalysis/Results/" + model_name + "/"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
      
    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,1)
        #Plot training & validation accuracy values
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Accuracy Vs Epochs')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Accuracy_FuzConvTrue_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Accuracy_FuzConvTrue_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Accuracy_FuzConvTrue_'+model_name+'.pdf', format='pdf')
        plt.show()
        
        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,2)
        #Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Loss Vs Epochs')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'Loss_FuzConvTrue_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'Loss_FuzConvTrue_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'Loss_FuzConvTrue_'+model_name+'.pdf', format='pdf')
        plt.show()

        '''
        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,2)
        #Plot training & validation loss values
        plt.plot(history.history['lr'])
        plt.ylabel('Lr')
        plt.xlabel('Epoch')
        plt.grid('on')
        plt.show()
        '''
    TIME = tm.time() - start_time
    print("Time "+model_name+" = %s [seconds]" % TIME)
    return {k:v for k,v in zip (model.metrics_names, metrics)}

In [7]:
def Final_Results_Test(model,PATH_trained_models):
    B_accuracy = 0 #B --> Best
    for filename in os.listdir(PATH_trained_models):
        if filename != ('train') and filename != ('validation'):
            print(filename)
            model.load_weights(PATH_trained_models+'/'+filename)
            loss,accuracy = model.evaluate(X_test, y_test,verbose=0)
            print(f'Loss={loss:.4f} y Accuracy={accuracy:0.4f}'+'\n') 
            if accuracy > B_accuracy:
                B_accuracy = accuracy
                B_loss = loss
                B_name = filename
    print("\n\nBest")
    print(B_name)
    print(f'Loss={B_loss:.4f} y Accuracy={B_accuracy:0.4f}'+'\n')

In [8]:
from tqdm import tqdm

def plot_train_valid(model,PATH_trained_models,model_name):
    acc_train=[]
    acc_valid=[]
    loss_train=[]
    loss_valid=[]
    for filename in tqdm(os.listdir(PATH_trained_models)):
        if filename != ('train') and filename != ('validation'):
            print(filename)
            model.load_weights(PATH_trained_models+'/'+filename)
            loss,accuracy = model.evaluate(X_train, y_train,verbose=0)
            acc_train.append(accuracy)
            loss_train.append(loss)
            loss,accuracy = model.evaluate(X_valid, y_valid,verbose=0)
            acc_valid.append(accuracy)
            loss_valid.append(loss)

    results_dir="/content/drive/My Drive/FuzConvSteganalysis/Results/"+model_name+"/"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    with plt.style.context('seaborn-white'):
        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,1)
        #Plot training & validation accuracy values
        plt.plot(acc_train)
        plt.plot(acc_valid)
        plt.title('Accuracy Vs Epochs')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.pdf', format='pdf')
        plt.show()

        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,2)
        #Plot training & validation loss values
        plt.plot(loss_train)
        plt.plot(loss_valid)
        plt.title('Loss Vs Epochs')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.grid('on')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.eps', format='eps')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.svg', format='svg')
        plt.savefig(results_dir+'FuzConvTrue_'+model_name+'.pdf', format='pdf')
        plt.show()

        '''
        plt.figure(figsize=(5, 5))
        #plt.subplot(1,2,2)
        #Plot training & validation loss values
        plt.plot(history.history['lr'])
        plt.ylabel('Lr')
        plt.xlabel('Epoch')
        plt.grid('on')
        plt.show()
        '''
    results={'acc_train':acc_train,'acc_valid':acc_valid,'loss_train':loss_train,'loss_valid':loss_valid}
    return results

## Plot ROC curves

In [9]:
from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)

def get_curve(gt, pred, target_names,model_name):
    labels=[]
    for i in range(len(target_names)):
        
        curve_function = roc_curve
        auc_roc = roc_auc_score(gt[:, i], pred[:, i])
        label = model_name+target_names[i] + " AUC: %.3f " % auc_roc
        labels.append(label)
        xlabel = "False positive rate"
        ylabel = "True positive rate"
        a, b, _ = curve_function(gt[:, i], pred[:, i])
        plt.figure(1, figsize=(7, 7))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.plot(a, b, label=label)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)

        plt.legend(loc='upper center', bbox_to_anchor=(1.3, 1),
                  fancybox=True, ncol=1)
      
    return [a,b],labels
labels = ["innocent","altered"]

## Working with BOSSbase 1.01 WOW PAYLOAD = 0.2bpp

In [10]:
PATH04 = '/content/drive/My Drive'
#Dataset
EPOCHS=100
PATH04_WOW1 = "/Numfiles/"

#Train
X_train = np.load(PATH04+PATH04_WOW1+'X_train.npy')
y_train = np.load(PATH04+PATH04_WOW1+'y_train.npy')
#Valid
X_valid = np.load(PATH04+PATH04_WOW1+'X_valid.npy')
y_valid = np.load(PATH04+PATH04_WOW1+'y_valid.npy')
#Test
X_test = np.load(PATH04+PATH04_WOW1+'X_test.npy')
y_test = np.load(PATH04+PATH04_WOW1+'y_test.npy')

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)


(46203, 256, 256)
(46203, 2)
(4297, 256, 256)
(4297, 2)
(5108, 256, 256)
(5108, 2)


## CNN name and algorithm 

In [11]:
base_name="FuzConvTrue"
m_name="FuzConvTrue"

## Training

In [ ]:
model= FuzConvTrue() 
name="Model_"+m_name+"_"+base_name
_, history  = train(model, X_train, y_train, X_valid, y_valid, X_test, y_test, batch_size=64, epochs=100, model_name=name)

## Test

In [ ]:
model= FuzConvTrue() 
PATH_trained_models = 'drive/MyDrive/FuzConvSteganalysis/trained_models'
Final_Results_Test(FuzConvTrue(),'drive/My Drive/FuzConvSteganalysis/trained_models/Model_FuzConvTrue_FuzConvTrue_1684741645.7035205')

## Training, validation and testing graph

In [ ]:
model= FuzConvTrue()  
PATH_trained_models = "drive/My Drive/FuzConvSteganalysis/trained_models/Model_FuzConvTrue_FuzConvTrue_1684741645.7035205"
name="Model_"+m_name+"_"+base_name
res=plot_train_valid(model,PATH_trained_models,name)

## ROC curves

In [ ]:
model= FuzConvTrue() 
model.load_weights("drive/My Drive/FuzConvSteganalysis/trained_models/Model_FuzConvTrue_FuzConvTrue_1684741645.7035205/saved-model-05-0.33.hdf5") #path best model
predictions= model.predict(X_test,verbose=0)
labels = ["innocent","altered"] 
model_name="FuzConvTrue "
curve1,labels1=get_curve(y_test, predictions, labels,model_name)

# END